<a href="https://colab.research.google.com/github/Moon1570/post-training-quantization/blob/main/4_bit_Quantize_Infinity_net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision monai


In [3]:
!git clone https://github.com/FoundationVision/Infinity.git
#!cd Infinity
#!pip install -r requirements.txt

fatal: destination path 'Infinity' already exists and is not an empty directory.


In [4]:
cd Infinity

/content/Infinity


In [5]:
!pip install -r requirements.txt

In [6]:
import os

# Create a directory for weights
os.makedirs("weights", exist_ok=True)

# ✅ Download Infinity-2B Model
!wget -O weights/infinity_2b_reg.pth https://huggingface.co/FoundationVision/infinity/resolve/main/infinity_2b_reg.pth

# ✅ Download Visual Tokenizer (VAE)
!wget -O weights/infinity_vae_d32reg.pth https://huggingface.co/FoundationVision/infinity/resolve/main/infinity_vae_d32reg.pth

# ✅ Install required libraries
!pip install torch torchvision torchaudio

--2025-02-16 03:43:59--  https://huggingface.co/FoundationVision/infinity/resolve/main/infinity_2b_reg.pth
Resolving huggingface.co (huggingface.co)... 3.165.160.59, 3.165.160.61, 3.165.160.11, ...
Connecting to huggingface.co (huggingface.co)|3.165.160.59|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: /FoundationVision/Infinity/resolve/main/infinity_2b_reg.pth [following]
--2025-02-16 03:43:59--  https://huggingface.co/FoundationVision/Infinity/resolve/main/infinity_2b_reg.pth
Reusing existing connection to huggingface.co:443.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/84/45/8445983988020af8980ce03b52cc249860ef6e4f4413e81ceac35347d4f7559e/d5958b37e7ecb914230cd2b8c28aae0ec108ce08b315720bc3dc52596cc5b3a7?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27infinity_2b_reg.pth%3B+filename%3D%22infinity_2b_reg.pth%22%3B&Expires=1739681039&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEY

In [7]:
!pip install auto-gptq bitsandbytes transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 98.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.9 MB/s eta 0:00:00


In [8]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, BitsAndBytesConfig

# ✅ Define model paths
text_encoder_ckpt = "google/flan-t5-xl"

# ✅ Enable 4-bit Quantization
quantization_config = BitsAndBytesConfig(load_in_4bit=True)

# ✅ Load tokenizer
text_tokenizer = AutoTokenizer.from_pretrained(text_encoder_ckpt)

# ✅ Load model with 4-bit quantization
text_encoder = AutoModelForSeq2SeqLM.from_pretrained(
    text_encoder_ckpt,
    device_map="auto",  # Auto-assign to GPU
    quantization_config=quantization_config
)

text_encoder.eval()
print("✅ Flan-T5-xl loaded successfully in 4-bit mode!")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.45G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

✅ Flan-T5-xl loaded successfully in 4-bit mode!


In [11]:
import torch
import argparse  # Ensure argparse is imported
from tools.run_infinity import *

# ✅ Define model paths
model_path = "weights/infinity_2b_reg.pth"
vae_path = "weights/infinity_vae_d32reg.pth"

# ✅ Define arguments for the model
args = argparse.Namespace(
    pn="1M",
    model_path=model_path,
    vae_path=vae_path,
    model_type="infinity_2b",
    vae_type=32,  # ✅ Ensure this is included
    apply_spatial_patchify=0,  # ✅ Add this line to fix the error
    bf16=1,  # Mixed precision support
)

# ✅ Load Infinity Model
vae = load_visual_tokenizer(args)  # Ensure VAE loads correctly
infinity = load_transformer(vae, args)

# Move model to GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
infinity.to(device)

print("✅ Infinity Model Loaded Successfully!")

AttributeError: 'Namespace' object has no attribute 'checkpoint_type'

In [ ]:
from auto_gptq import AutoGPTQForCausalLM

# ✅ Load Infinity model in GPTQ format
quantized_model = AutoGPTQForCausalLM.from_pretrained(
    model_path,
    use_triton=False,  # Set to True if you want to use Nvidia Triton
    quantize_config={"bits": 4}  # GPTQ 4-bit Quantization
)

# ✅ Move model to GPU
quantized_model.to(device)

# ✅ Save Quantized Model
torch.save(quantized_model.state_dict(), "weights/infinity_gptq_4bit.pth")
print("✅ Quantized Infinity Model Saved Successfully!")

In [ ]:
# ✅ Load Quantized Model for Inference
quantized_model = AutoGPTQForCausalLM.from_pretrained(
    model_path,
    quantize_config={"bits": 4}
)
quantized_model.to(device)

print("✅ Quantized Infinity Model Ready for Inference!")